In [11]:
import fsspec
import os 
import tarfile
from datetime import datetime, timedelta

In [ ]:
"""
This is the same basis as the original radar download script I made, but instead 
of using the dumpster fire that is AWS, and instead uses google cloud. 
This is orders of magnitude faster than the AWS script.

Make sure to run "pip install gcsfs" before using this script!

""" 

# Set up the GCS filesystem (anonymous access)
fs = fsspec.filesystem("gcs", token=None)  # `token=None` enables anonymous access

def get_user_input():
    start_date = input("Enter start date and time (YYYY-MM-DD HH:MM): ").strip()
    end_date = input("Enter end date and time (YYYY-MM-DD HH:MM): ").strip()
    station = input("Enter radar station code (e.g., KARX): ").strip().upper()
    download_dir = input("Enter directory to save radar files: ").strip()

    return start_date, end_date, station, download_dir
    

def extract_tar_file(tar_path, extract_dir):
    """
    Extracts the contents of a .tar file into the specified directory
    and removes any files ending with 'MDM.ar2v'.
    """
    try:
        with tarfile.open(tar_path, "r") as tar:
            tar.extractall(path=extract_dir)  # Extract all files first

        print(f"Extracted: {tar_path} -> {extract_dir}")

        # Remove files ending with 'MDM.ar2v'
        for root, dirs, files in os.walk(extract_dir):
            for file in files:
                if file.endswith("MDM.ar2v"):
                    full_path = os.path.join(root, file)
                    os.remove(full_path)
                    print(f"Removed unwanted file: {full_path}")

    except Exception as e:
        print(f"Failed to extract {tar_path}: {e}")

def download_radar_files(start_date_str, end_date_str, station, download_dir):
    # Convert string dates to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M')

    # Ensure the download directory exists
    os.makedirs(download_dir, exist_ok=True)

    # Generate the list of files for the specified date range
    current_time = start_date
    files_downloaded = 0

    while current_time <= end_date:
        yr = current_time.strftime("%Y")
        mo = current_time.strftime("%m")
        dy = current_time.strftime("%d")
        hr = current_time.strftime("%H")

        # Construct the search pattern
        search_pattern = (
            f"gcs://gcp-public-data-nexrad-l2/{yr}/{mo}/{dy}/{station}/"
            f"NWS_NEXRAD_NXL2DPBL_{station}_{yr}{mo}{dy}{hr}*"
        )

        all_files = fs.glob(search_pattern)
        filtered_files = [f for f in all_files if f.endswith(".tar")]

        for file in filtered_files:
            local_filename = os.path.join(download_dir, os.path.basename(file))
            
            try:
                # Download the file
                with fs.open(file, "rb") as f_in, open(local_filename, "wb") as f_out:
                    f_out.write(f_in.read())
                print(f"Downloaded {local_filename}")
                files_downloaded += 1

                # Extract the .tar file directly into download_dir
                extract_tar_file(local_filename, download_dir)

                # Optionally, delete the .tar file after extraction
                os.remove(local_filename)
                print(f"Deleted archive: {local_filename}")

            except Exception as e:
                print(f"Failed to download {file}: {e}")

        # Move forward hour by hour
        current_time += timedelta(hours=1)

    print(f"Download completed. Total files downloaded: {files_downloaded}")

if __name__ == "__main__":
    start_date_str, end_date_str, station, download_dir = get_user_input()
    download_radar_files(start_date_str, end_date_str, station, download_dir)
